# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
weather_df = pd.read_csv('../output_data/Weather_Output.csv')

In [41]:
weather_df.head(1)

,Unnamed: 0,City,country,Latitude,Longitude,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,atuona,PF,-9.8,-139.03,1.583964e+09,82.18,77.0,15.0,11.12


In [53]:
weather_df = weather_df[["City","country","Latitude" ,"Longitude", "Max Temp", "Humidity", "Cloudiness","Wind Speed"]]
weather_df.head()

,City,country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,atuona,PF,-9.80,-139.03,82.18,77.0,15.0,11.12
1,mataura,NZ,-46.19,168.86,48.38,85.0,44.0,3.04
2,harnosand,SE,62.63,17.94,33.01,98.0,20.0,3.36
3,hermanus,ZA,-34.42,19.23,70.00,84.0,99.0,8.99
4,hobart,AU,-42.88,147.33,59.00,72.0,29.0,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [54]:
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"].astype(float)

In [55]:
print(locations)

     Latitude  Longitude
0       -9.80    -139.03
1      -46.19     168.86
2       62.63      17.94
3      -34.42      19.23
4      -42.88     147.33
..        ...        ...
541    -24.13      19.12
542     33.64     -96.61
543     -8.23     -78.97
544    -17.64     -71.34
545     60.87      60.42

[546 rows x 2 columns]


In [57]:
gmaps.configure(api_key=g_key )


fig = gmaps.figure(center=(10.0,-5.0), zoom_level=2.25)


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=2)


fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
weather_fit_df = weather_df.loc[(weather_df["Max Temp"]>70) & (weather_df["Max Temp"]<85) & (weather_df["Wind Speed"]<50)
                                 & (weather_df["Cloudiness"]==0)]
weather_fit_df = weather_fit_df.dropna()
weather_fit_df.head()

,City,country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
26,cidreira,BR,-30.18,-50.21,75.76,84.0,0.0,15.88
79,castro,BR,-24.79,-50.01,72.82,63.0,0.0,2.82
90,port blair,IN,11.67,92.75,79.16,68.0,0.0,6.78
95,rawson,AR,-43.30,-65.10,70.47,56.0,0.0,0.27
111,sao filipe,CV,14.90,-24.50,74.95,59.0,0.0,16.28


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
hotel_df = weather_fit_df

params = {
    "radius": 5000,
    "types": "Hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]

    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

  
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
   
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 26: cidreira.
Closest hotel is CIDREIRA.
------------
Retrieving Results for Index 79: castro.
Closest hotel is Castro.
------------
Retrieving Results for Index 90: port blair.
Closest hotel is Port Blair.
------------
Retrieving Results for Index 95: rawson.
Closest hotel is Rawson.
------------
Retrieving Results for Index 111: sao filipe.
Closest hotel is Sao Filipe.
------------
Retrieving Results for Index 121: poum.
Closest hotel is Poum.
------------
Retrieving Results for Index 162: wagar.
Closest hotel is Wagga.
------------
Retrieving Results for Index 168: salalah.
Closest hotel is Salalah.
------------
Retrieving Results for Index 169: arraial do cabo.
Closest hotel is Cabo Frio.
------------
Retrieving Results for Index 346: aden.
Closest hotel is Aden.
------------
Retrieving Results for Index 363: pisco.
Closest hotel is Pisco.
------------
Retrieving Results for Index 368: raga.
Closest hotel is Raga.
------------
Retrieving Results for Ind

In [48]:

hotel_df.head()

,City,country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
26,cidreira,BR,-30.18,-50.21,75.76,84.0,0.0,15.88,CIDREIRA
79,castro,BR,-24.79,-50.01,72.82,63.0,0.0,2.82,Castro
90,port blair,IN,11.67,92.75,79.16,68.0,0.0,6.78,Port Blair
95,rawson,AR,-43.30,-65.10,70.47,56.0,0.0,0.27,Rawson
111,sao filipe,CV,14.90,-24.50,74.95,59.0,0.0,16.28,Sao Filipe


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [50]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))